In [ ]:
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt
import matplotlib

In [ ]:
import matplotlib.pyplot as plt

def set_share_axes(axs, target=None, sharex=False, sharey=False):
    if target is None:
        target = axs.flat[0]
    # Manage share using grouper objects
    for ax in axs.flat:
        if sharex:
            target._shared_axes['x'].join(target, ax)
        if sharey:
            target._shared_axes['y'].join(target, ax)
    # Turn off x tick labels and offset text for all but the bottom row
    if sharex and axs.ndim > 1:
        for ax in axs[:-1,:].flat:
            ax.xaxis.set_tick_params(which='both', labelbottom=False, labeltop=False)
            ax.xaxis.offsetText.set_visible(False)
    # Turn off y tick labels and offset text for all but the left most column
    if sharey and axs.ndim > 1:
        for ax in axs[:,1:].flat:
            ax.yaxis.set_tick_params(which='both', labelleft=False, labelright=False)
            ax.yaxis.offsetText.set_visible(False)

In [ ]:
palette = sns.color_palette()
color_matching = {
    "ELEET": palette[0],  # blue
    "Text-To-Table": palette[2],  # green
    "LLaMA-2 7B (ic)": palette[1],  # orange
    "LLaMA-2 7B (ft)": palette[3],  # red
    "gpt-3.5-turbo-0125": palette[4],  #purple
    "gpt-4-0613": palette[6]
}
methods = ["ELEET", "Text-To-Table", "LLaMA-2 7B (ic)", "LLaMA-2 7B (ft)", "gpt-3.5-turbo-0125", "gpt-4-0613"]
color_palette = [color_matching[m] for m in methods]

In [ ]:
from eleet.evaluate import RunDescription

METRICS_FILE = "../predictions/test/metrics_eleet-t2t-llama-gpt-3.5-turbo-0125-gpt-4-0613-llama-ft-gpt-ft-rotowire-trex-aviation-corona-64-test-limit-4294967296.csv"
RUNTIME_FILE = "../predictions/test/runtimes_eleet-t2t-llama-gpt-3.5-turbo-0125-gpt-4-0613-llama-ft-gpt-ft-rotowire-trex-aviation-corona-64-test-limit-4294967296.csv"
PERF_METRIC = "mean_f1"

metrics = pd.read_csv(METRICS_FILE)
runtime = pd.read_csv(RUNTIME_FILE)

runtime["method"] = runtime["method"].apply(lambda x: "gpt-3.5-turbo (ft)" if x.startswith("ft:") else x)
metrics["method"] = metrics["method"].apply(lambda x: "gpt-3.5-turbo (ft)" if x.startswith("ft:") else x)
runtime["runtime"] = runtime["runtime"] - runtime["index_build_time"]
best_runtime = runtime.groupby("query")["runtime"].agg(min)
best_runtime.name = "best runtime"
best_perf = metrics.groupby("query")[PERF_METRIC].agg(max)
best_perf.name = "best perf"

merged = metrics.merge(runtime, on=RunDescription._fields).merge(best_runtime, on="query").merge(best_perf, on="query")
merged["slowdown"] = merged["runtime"] / merged["best runtime"]
merged["mean F1 decrease"] = merged["best perf"] / merged[PERF_METRIC]

merged.replace({"ELEET": "ELEET", "LLaMA": "LLaMA-2 7B (ic)", 4096: 3398, 16384: 3398, "LLaMA-FT": "LLaMA-2 7B (ft)"}, inplace=True)

merged["is_join"] = merged["query"].apply(lambda x: "⨝" in x)
merged["is_union"] = merged["query"].apply(lambda x: "∪" in x)
merged["is_scan"] = merged["query"].apply(lambda x: "⨝" not in x and "∪" not in x)
merged["is_aggregation"] = merged["query"].apply(lambda x: "G_" in x)
merged["is_selection"] = merged["query"].apply(lambda x: "[" in x)
merged["type"] = merged.apply(lambda x: next(y for y in ["aggregation", "selection", "scan", "union", "join"]
                                             if x[f"is_{y}"]), axis=1)

# only nobel is suitable for evaluation
merged = merged[~(merged["dataset"] == "trex") | (merged["query"].apply(lambda x: "Personal" in x or "Career" in x))]



# matplotlib.rcParams.update({'font.size': })
plt.rc('text', usetex=False)
sns.set_theme(style="whitegrid", font_scale=1.0)


fig = plt.figure(figsize=(16, 2.5))
gs = fig.add_gridspec(nrows=2, ncols=5, hspace=0.05, wspace=0.05, height_ratios=[1, 3],
                      width_ratios=[6, 6, 6, 6, 1]) #, "wspace": 0.1})
axes = gs.subplots()
set_share_axes(axes[:,:4], sharex=True)
set_share_axes(axes[1,:], sharey=True)

labels = list()
handles = list()

for i, dataset in enumerate(("rotowire", "trex", "aviation", "corona")):
    ax_density = axes[0, i]
    ax_scatter = axes[1, i]

    sns.kdeplot(merged[merged["dataset"] == dataset], x="mean_f1", hue="method", ax=ax_density,
                hue_order=methods[::-1], palette=sns.color_palette(color_palette[::-1]))
    sns.scatterplot(merged[merged["dataset"] == dataset], y="runtime", x="mean_f1", hue="method", ax=ax_scatter,
                    hue_order=methods[::-1], palette=sns.color_palette(color_palette[::-1])
                    # style="type", style_order=["aggregation", "selection", "scan", "union", "join"]
                    )

    for ax in (ax_density, ax_scatter):
        handles_ax, labels_ax = ax.get_legend_handles_labels()
        handles.extend(handles_ax)
        labels.extend(labels_ax)
        ax.get_legend().remove()

    ax_density.set_title(dataset)
    ax_density.set_yticks([])
    ax_density.set_ylabel("Density\nof\nMean F1" if i == 0 else "", fontsize=12)
    ax_density.get_yaxis().set_label_coords(-0.1, 0.7)
    # ax_density.get_xaxis().set_visible(False)
    ax_scatter.set_ylabel("")

    if i == 0:
        ax_scatter.set_yscale("log")
        ax_scatter.set_ylabel("Runtime [s]", fontsize=12)
        ax_scatter.set_xlim((-0.1, 1.1))
        ax_scatter.set_ylim((5, 50_000))
        ax_scatter.invert_yaxis()

    ax_scatter.set_xlabel("Mean F1", fontsize=12)
    ax_scatter.set_yticks([10, 100, 1000, 10000])
    ax_scatter.set_yticklabels([10, 100, 1000, 10000])
    ax_scatter.set_xticks([0, 0.25, 0.5, 0.75, 1], fontsize=12)
    ax_scatter.set_xticklabels([0, 0.25, 0.5, 0.75, 1], fontsize=12)

    if i != 0:
        ax_scatter.set_yticklabels(["", "", "", ""], fontsize=12)


axes[0,4].set_axis_off()
sns.kdeplot(merged, y="runtime", hue="method", ax=axes[1,4],
            hue_order=methods[::-1], palette=sns.color_palette(color_palette[::-1]))
axes[1,4].set_yticks([10, 100, 1000, 10000])
axes[1,4].set_yticklabels(["", "", "", ""])
axes[1,4].set_xticks([])
axes[1,4].get_legend().remove()
axes[1,4].set_xlabel("Density\nof\nRuntime")
axes[1,4].set_ylabel("")

hl_dict = {l.split("_")[0]: h for l, h in zip(labels, handles)}
print(hl_dict)
leg = fig.legend([hl_dict[l] for l in methods], methods, bbox_to_anchor=(0.9, 0.1), ncol=1, loc='lower left', borderaxespad=0., frameon=False)

plt.savefig("/home/murban/exp1.pdf", bbox_inches="tight")
plt.show()


In [ ]:
RUNTIME_FILE_FEW_WORKERS = "../predictions-few-workers/test/runtimes_eleet-rotowire-64-test-limit-4294967296.csv"

min_runtime = merged.groupby("query")["runtime"].min()
max_runtime = merged.groupby("query")["runtime"].max()

multiplier = (max_runtime / min_runtime).max()
q_id = (max_runtime / min_runtime).argmax()
query = max_runtime.index[q_id]


In [ ]:
multiplier

In [ ]:
query_df = merged[merged["query"] == query]
pd.concat((query_df.iloc[query_df["runtime"].argmax()], query_df.iloc[query_df["runtime"].argmin()]), axis=1)